# ETL Google Maps

## Metadata

In [1]:
# IMporto las librerias necesarias
import pandas as pd
import numpy as np
import re
import random
from nltk.corpus import stopwords
from textblob import TextBlob

In [2]:
# Abrimos el archivo metadata1.json
df = pd.read_json('C:/Users/dines/Desktop/Proyecto_final/metadata-sitios/metadata1.json', lines=True)

In [3]:
# Visualizo el df
df.head()

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.357100,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.215290,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.292130,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
3,Nova Fabrics,"Nova Fabrics, 2200 E 11th St, Los Angeles, CA ...",0x80c2c89923b27a41:0x32041559418d447,None,34.023669,-118.232930,[Fabric store],3.3,6,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...","{'Service options': ['In-store shopping'], 'Pa...",Open ⋅ Closes 5PM,"[0x80c2c8811477253f:0x23a8a492df1918f7, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
4,Nobel Textile Co,"Nobel Textile Co, 719 E 9th St, Los Angeles, C...",0x80c2c632f933b073:0xc31785961fe826a6,None,34.036694,-118.249421,[Fabric store],4.3,7,None,"[[Thursday, 9AM–5PM], [Friday, 9AM–5PM], [Satu...",{'Service options': ['In-store pickup']},Open ⋅ Closes 5PM,"[0x80c2c62c496083d1:0xdefa11317fe870a1, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [4]:
# Observamos las columnas la cantidad de nulos y el tipo de dato de cada columna
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275001 entries, 0 to 275000
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   name              274994 non-null  object 
 1   address           264939 non-null  object 
 2   gmap_id           275001 non-null  object 
 3   description       13155 non-null   object 
 4   latitude          275001 non-null  float64
 5   longitude         275001 non-null  float64
 6   category          272740 non-null  object 
 7   avg_rating        275001 non-null  float64
 8   num_of_reviews    275001 non-null  int64  
 9   price             13450 non-null   object 
 10  hours             192448 non-null  object 
 11  MISC              194972 non-null  object 
 12  state             195523 non-null  object 
 13  relative_results  238771 non-null  object 
 14  url               275001 non-null  object 
dtypes: float64(3), int64(1), object(11)
memory usage: 31.5+ MB


In [5]:
# Observamos el tamaño del df
df.shape

(275001, 15)

In [6]:
# Elimino id diplicados
df.drop_duplicates(subset='gmap_id', inplace=True)

In [7]:
#Creo columna city
df['city'] = df['address'].str.split(',').str[-2]

In [8]:
# Separo el postal code de la columna address    
df['address'] = df['address'].str.split(',').str[-1]
df['postal_code'] = df['address'].str.slice(-5)
# Cambio el tipo de dato de la columna address
df['postal_code'] = pd.to_numeric(df['postal_code'], errors='coerce')
df['postal_code'] = df['postal_code'].dropna()
df['postal_code'] = df['postal_code'].dropna().astype(int)
# Filtro por el codigo postal de florida 32003 hasta 34997
df = df[(df['postal_code'] >= 32003) & (df['postal_code'] <= 34997)]

In [9]:
# Hago una lista con las categorias que necesito
categorias = ['restaurant', 'restaurants', 'fast food', 'fastfood', 'hamburger', 'pizza', 'sandwich']
# Función para filtrar la categoria
def buscar_coincidencias(lista):
    if lista is None:
        return False
    else:
        texto = ' '.join(lista).lower()
        for palabra in categorias:
            if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto):
                return True
        return False

In [10]:
# Verifico si alguna de las categorias esta en la columna 'category'
df['categorias'] =  df['category'].apply(buscar_coincidencias)
# Filtro el dfme quedo con las que se encontro resultado en la nueva columna
df = df[df['categorias']]

In [11]:
# Obtengo la tabla city y creo el city_id, ordeno el df por orden alfabetico y reseteo el indice
city = df['city'].drop_duplicates().sort_values().reset_index()


In [12]:
# Relleno la columna city_id con el id unico
city['city_id'] = '000' + city.index.astype(str)
city['city_id'] = city['city_id'].str[-4:]

In [13]:
# Elimino l columna index
city.drop(columns='index', inplace = True)

In [14]:
# Observo la estructura del df city
city.head()

,city,city_id
0,Alva,0000
1,Anna Maria,0001
2,Apalachicola,0002
3,Auburndale,0003
4,Aventura,0004


In [15]:
# Creo un path con cada ruta y una lista de paths, para pasar por la funcion y concatenar todos los df
path1 = 'metadata-sitios/metadata1.json' 
path2= 'metadata-sitios/metadata2.json'
path3 = 'metadata-sitios/metadata3.json'
path4 = 'metadata-sitios/metadata4.json'
path5 = 'metadata-sitios/metadata5.json'
path6 = 'metadata-sitios/metadata6.json'
path7 = 'metadata-sitios/metadata7.json'
path8 = 'metadata-sitios/metadata8.json'
path9 = 'metadata-sitios/metadata9.json'
path10 = 'metadata-sitios/metadata10.json'
path11 = 'metadata-sitios/metadata11.json'

paths = [path1, path2, path3, path4,
         path5, path6, path7, path8,
         path9, path10, path11]


In [16]:
# Función creada a partir del análisis anterior, para poder limpiar todos los df metadata y unirlos en un solo csv
def process_metadata_gm(paths):
    result = []
    for path in paths:
        df = pd.read_json(path, lines=True)
        # Elimino id diplicados
        df.drop_duplicates(subset='gmap_id', inplace=True)
        #Creo columna city
        df['city'] = df['address'].str.split(',').str[-2]
        # Separo el postal code de la columna address    
        df['address'] = df['address'].str.split(',').str[-1]
        df['postal_code'] = df['address'].str.slice(-5)
        # Cambio el tipo de dato de la columna address
        df['postal_code'] = pd.to_numeric(df['postal_code'], errors='coerce')
        df['postal_code'] = df['postal_code'].dropna()
        df['postal_code'] = df['postal_code'].dropna().astype(int)
        # Filtro por el codigo postal de florida 32003 hasta 34997
        df = df[(df['postal_code'] >= 32003) & (df['postal_code'] <= 34997)]
        # Hago una lista con las categorias que necesito
        categorias = ['restaurant', 'restaurants', 'fast food', 'fastfood', 'hamburger', 'pizza', 'sandwich']
        # Función para filtrar la categoria
        def buscar_coincidencias(lista):
            if lista is None:
                return False
            else:
                texto = ' '.join(lista).lower()
                for palabra in categorias:
                    if re.search(r'\b' + re.escape(palabra.lower()) + r'\b', texto):
                        return True
                return False
        
        # Verifico si alguna de las categorias esta en la columna 'category'
        df['categorias'] =  df['category'].apply(buscar_coincidencias)
        # Filtro el dfme quedo con las que se encontro resultado en la nueva columna
        df = df[df['categorias']]
        # Elimino columnas innecesarias
        columns = ['description', 'address', 'price', 'postal_code', 'hours', 'MISC', 'state', 'relative_results', 'url', 'categorias'] 
        df.drop(columns, axis=1, inplace=True)
        # Agruega df a la lista de resultados
        result.append(df)
    # Uno los DF verticalmente
    merged_dfs = pd.concat(result, axis=0)
    # Reinicio el indice
    merged_dfs.reset_index(drop=True, inplace=True)
    # Reordeno las columnas
    columns= ['gmap_id', 'name', 'city', 'postal_code', 'latitude', 'longitude', 'category', 'avg_rating', 'num_of_reviews']
    merged_dfs = merged_dfs[columns]
    # Creo la tabla city y la columna city_id
    city = df['city'].drop_duplicates().reset_index()
    city['city_id'] = '000' + city.index.astype(str)
    city['city_id'] = city['city_id'].str[-4:]
    city = city[['city_id', 'city']]
    merged_dfs.to_csv('metadata-sitios/business.csv', index=False, encoding='utf-8')
    print('Archivos csv guardados con éxito.')
    return merged_dfs

## Review-Florida

In [22]:
# Abrimos el archivo review
df = pd.read_json('C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL1.json', lines=True)

In [23]:
# Visualizo la estructura del df
df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,"{'time': 1627042799532, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,"{'time': 1582464056733, 'text': 'Thank you for...",0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb


In [24]:
# Visualizo las columnas los nulos y el tipo de dato
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  150000 non-null  float64
 1   name     150000 non-null  object 
 2   time     150000 non-null  int64  
 3   rating   150000 non-null  int64  
 4   text     100023 non-null  object 
 5   pics     4577 non-null    object 
 6   resp     31967 non-null   object 
 7   gmap_id  150000 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 9.2+ MB


In [25]:
# Selecciono las columnas a utilizar
df = df[['user_id', 'name', 'time', 'rating', 'text', 'gmap_id']]

In [26]:
# Cambio el formato de la fecha y observo el nuevo formato
df['time'] = pd.to_datetime(df['time'], unit='ms').dt.date
df['time'].head()

0    2021-08-03
1    2020-07-18
2    2018-04-05
3    2016-07-07
4    2016-12-02
Name: time, dtype: object

In [27]:
# Verifico fechas maxima y minima
print(df['time'].max())
print(df['time'].min())

2021-09-09
2003-05-11


In [28]:
# reviso cuantos registro hay desde 2017
df[df['time'].apply(lambda x: x.year)>=2017].shape

(138935, 6)

In [29]:
# Filtro por los business
df_rest = pd.read_csv('C:/Users/dines/Desktop/Proyecto_final/metadata-sitios/business.csv')
# Creo una lista con los business        
list_id = list(df_rest['gmap_id'])

In [30]:
# Visualizo la lista de id
list_id

['0x88db4147b1d9e6f3:0x943dbd10a92ba1b1',
 '0x88c2e4e34f1ed783:0x76c5da381c499d79',
 '0x8890b9241e704667:0x3a1e565c17c00993',
 '0x8894b5a7a7909725:0xe4687299fd288188',
 '0x88e635378f43352f:0xa1b53c63436fa428',
 '0x88c2fea614c52c8b:0x50e166caa05e8c0f',
 '0x88e8a351be3b7517:0xfc2c0c945c5221ff',
 '0x88d9ab98b5baad79:0x1a2e3f0642a55246',
 '0x88d9b86c23bb04eb:0x2a82da546b7feb1b',
 '0x88d9ba5d65937567:0xbc27649cf513cc89',
 '0x88d900575f0dd065:0x9b3638d2a80be4d',
 '0x88e765a15b829793:0x34addf6ffa1d3f77',
 '0x88d9b719170a9f61:0x30d454a980f76ad3',
 '0x88d769c187b52953:0xe3ace06a033182f6',
 '0x88e76652cd84272f:0x548abb9935d912ff',
 '0x88dd83d661b9262f:0xbc3de0d8abe6423b',
 '0x88d9b395c6179c89:0x914b0aafb453b3b5',
 '0x88d91d9e5e488c4f:0x6049fa30d0806848',
 '0x146c6711a70d7991:0xe478335f760c8ccc',
 '0x88d9b1f38dff4dff:0x61443de98cd9c88c',
 '0x88c2c5c515fa6e53:0xee4f630ca314fe01',
 '0x88dd8019b7c4167b:0x7043f86a8346dcf3',
 '0x88e76d9181abf2a9:0x97194d850024543c',
 '0x88dd7f02de6ab8af:0xbab9000f15a5

In [31]:
#Filtro por gmap_id
florida_rest = df[df['gmap_id'].isin(list_id)]

In [32]:
# Visualizo la estructura del nuevo df
florida_rest.head()

,user_id,name,time,rating,text,gmap_id
514,1.130683e+20,Edwin Ashurst,2018-09-17,5,The salads were lackluster though with well wr...,0x8894b5a7a7909725:0xe4687299fd288188
515,1.059416e+20,Douglas Gummere,2018-09-20,1,"Love this place. Great little restaurant, with...",0x8894b5a7a7909725:0xe4687299fd288188
516,1.098577e+20,David Powell,2018-08-22,5,A classy restaurant to match the charm of the ...,0x8894b5a7a7909725:0xe4687299fd288188
517,1.147377e+20,Hillary Hutchins,2019-04-17,5,Great atmosphere and good food,0x8894b5a7a7909725:0xe4687299fd288188
518,1.089870e+20,Ajay patel,2019-05-20,5,Good food,0x8894b5a7a7909725:0xe4687299fd288188


In [33]:
# Reviso duplicados
duplicados = florida_rest.duplicated(keep=False)
florida_rest[duplicados].sort_values('user_id', ascending=True)

,user_id,name,time,rating,text,gmap_id
18686,1.000209e+20,Dee Gee,2017-01-21,5,"Great tasting food, all the info you need is o...",0x88d907944065b7cb:0xc3c8f9877b52c59e
8863,1.000209e+20,Dee Gee,2017-01-21,5,"Great tasting food, all the info you need is o...",0x88d907944065b7cb:0xc3c8f9877b52c59e
3678,1.000703e+20,cristian garcia,2018-01-10,4,Nice place food is good price decent just hope...,0x88dd738fd4ebbe91:0xe61f34790c936cc0
13501,1.000703e+20,cristian garcia,2018-01-10,4,Nice place food is good price decent just hope...,0x88dd738fd4ebbe91:0xe61f34790c936cc0
46874,1.001137e+20,rajiv roopchand,2020-03-03,1,Online orders don't work here,0x88d9012935620241:0xb3a9b5b0671e51a5
...,...,...,...,...,...,...
1619,1.183783e+20,Mary Ann Kapostasy,2020-02-27,4,Food was delicious,0x88d769c187b52953:0xe3ace06a033182f6
11594,1.184306e+20,Mannuel Aguirre,2020-02-02,5,Great BBQ.,0x88d9b395c6179c89:0x914b0aafb453b3b5
1771,1.184306e+20,Mannuel Aguirre,2020-02-02,5,Great BBQ.,0x88d9b395c6179c89:0x914b0aafb453b3b5
3688,1.184360e+20,Emmanuel García,2020-11-12,5,None,0x88dd738fd4ebbe91:0xe61f34790c936cc0


In [34]:
# Elimino duplicados
index_duplicated = duplicados[duplicados].index
florida_rest.drop(index_duplicated, inplace=True)

C:\Users\dines\AppData\Local\Temp\ipykernel_11884\1600229536.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  florida_rest.drop(index_duplicated, inplace=True)


In [35]:
# Genero los path con la ruta de cada archivo y armo una lista con todos los paths
path1 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL1.json' 
path2= 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL2.json'
path3 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL3.json'
path4 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL4.json'
path5 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL5.json'
path6 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL6.json'
path7 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL7.json'
path8 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL8.json'
path9 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL9.json'
path10 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL10.json'
path11 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL11.json'
path12 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL12.json'
path13 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL13.json'
path14 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL14.json'
path15 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL15.json'
path16 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL16.json'
path17 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL17.json'
path18 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL18.json'
path19 = 'C:/Users/dines/Desktop/Proyecto_final/review_Florida/reviewFL19.json'

paths = [path1, path2, path3, path4,
         path5, path6, path7, path8,
         path9, path10, path11, path12,
         path13, path14, path15, path16, 
         path17, path18, path19]

* A partir del análisis anterior construmos esta funcion que filtra todos los df y los contatena en uno.
* A demas incorporamos un análisis de sentimiento generando una valuacion en una nueva columna

In [41]:
def filtro_review(paths):
    result = []
    for path in paths:
        df = pd.read_json(path, lines=True)
        # Selecciono las columnas que voy a utilizar
        df = df[['user_id', 'name', 'time', 'rating', 'text', 'gmap_id']]
        # Cambio el tipo de fecha
        df['time'] = pd.to_datetime(df['time'], unit='ms').dt.date
        # Filtro registros a partir de 2017(últimos 5 años)
        df=df[df['time'].apply(lambda x: x.year)>=2017]
        # Cargo el csv business
        df_rest = pd.read_csv('C:/Users/dines/Desktop/Proyecto_final/ETL_final/business.csv')
        # Obtengo una lista con los gmap_id
        list_id = list(df_rest['gmap_id'])
        #Filtro por gmap_id
        df = df[df['gmap_id'].isin(list_id)]
        # Agrego el df a la lista
        result.append(df)
    # Uno los DF verticalmente
    merged_dfs = pd.concat(result, axis=0)
    # Reinicio el indice
    merged_dfs.reset_index(drop=True, inplace=True)
    # Elimino duplicados
    duplicados = merged_dfs.duplicated(keep='first')
    indice_duplicado = duplicados[duplicados].index
    merged_dfs.drop(indice_duplicado, inplace=True)
    merged_dfs.reset_index(drop=True, inplace=True)
    # Análisis de sentimiento
    # Elimino proposiciones y emojis
    stop_words = set(stopwords.words('english'))
    merged_dfs['text'] = merged_dfs['text'].astype(str).apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
    merged_dfs['text'] = merged_dfs['text'].apply(lambda x: re.sub(r'[^/w/s]', '', x))
    # Limpio los datos
    merged_dfs['text'] = merged_dfs['text'].str.lower()
    merged_dfs['text'] = merged_dfs['text'].str.replace('[^a-zA-Z0-9]', ' ')
    # Aplico análisis de sentimiento
    merged_dfs['sentiment'] = merged_dfs['text'].apply(lambda x: TextBlob(x).sentiment.polarity)
    # Creo la columna 'sentiment_analysis'
    merged_dfs['sentiment_analysis'] = merged_dfs['sentiment'].apply(lambda x: 0 if x < 0 else 1 if x == 0 else 2)
    # Elimino columna sentiment
    merged_dfs = merged_dfs.drop(columns='sentiment', axis=1)
    # Reordeno las columnas
    columnas = ['user_id', 'name', 'time', 'rating', 'text', 'sentiment_analysis', 'gmap_id']
    merged_dfs = merged_dfs[columnas]
    
    load_path= "C:/Users/dines/Desktop/Proyecto_final/ETL_final/reviewfinal.csv"
    merged_dfs.to_csv(load_path, index=False, encoding="utf-8")
    print(f"Archivo CSV guardado con éxito") 

In [42]:
florida_reviews = filtro_review(paths)

Archivo CSV guardado con éxito


# Users

In [45]:
florida = pd.read_csv('C:/Users/dines/Desktop/Proyecto_final/ETL_final/reviewfinal.csv')


In [46]:
# Selecciono las columnas que voy a utilizar
df_user = florida[['user_id', 'name']]

In [47]:
# ELimino duplicados
df_user['user_id'].drop_duplicates()
df_user

,user_id,name
0,1.130683e+20,Edwin Ashurst
1,1.059416e+20,Douglas Gummere
2,1.098577e+20,David Powell
3,1.147377e+20,Hillary Hutchins
4,1.089870e+20,Ajay patel
...,...,...
235765,1.111539e+20,Monica B
235766,1.058293e+20,Latonshia Smith
235767,1.148710e+20,Alexander Dunn
235768,1.154891e+20,ACT Tech


In [48]:
# Guardo el csv
df_user.to_csv('C:/Users/dines/Desktop/Proyecto_final/ETL_final/df_user.csv')